In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


UsageError: %%python is a cell magic, but the cell body is empty.


In [2]:
test_data = pd.read_csv('../input/titanic/test.csv')
train_data = pd.read_csv('../input/titanic/train.csv')
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [3]:
y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch", "Embarked", "Age", "Fare"]
X = pd.get_dummies(train_data[features])
X.head(3)

,Pclass,SibSp,Parch,Age,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,1,0,22.0,7.2500,0,1,0,0,1
1,1,1,0,38.0,71.2833,1,0,1,0,0
2,3,0,0,26.0,7.9250,1,0,0,0,1


In [4]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42)

# find columns with missing value
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

In [5]:
# fill missing value with imputed value
imputer = SimpleImputer()

X_train_copy = X_train.copy()
X_valid_copy = X_valid.copy()

for col in cols_with_missing:
    X_train_copy[col + '_was_missing'] = X_train_copy[col].isnull()
    X_valid_copy[col + '_was_missing'] = X_valid_copy[col].isnull()

imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train_copy))
imputed_X_valid = pd.DataFrame(imputer.transform(X_valid_copy))

imputed_X_train.columns = X_train_copy.columns
imputed_X_valid.columns = X_valid_copy.columns

In [6]:
# Drop columns, with a missing value, in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

In [7]:
score_drop_na = DecisionTreeClassifier(random_state=42).fit(reduced_X_train,y_train).score(reduced_X_valid, y_valid)
score_impute = DecisionTreeClassifier(random_state=42).fit(imputed_X_train,y_train).score(imputed_X_valid, y_valid)

print('score drop missing value :', score_drop_na)
print('score impute value:', score_impute)

# droping missing column entirely (Age) seems to perform
# better than trying to fill those missing value

score drop missing value : 0.7694915254237288
score impute value: 0.7559322033898305


### Finding optimal parameters for tree and knn

In [8]:
cols_with_missing

['Age']

In [9]:
features = ["Pclass", "Sex", "SibSp", "Parch", "Embarked", "Fare"]
X = pd.get_dummies(train_data[features])

y = train_data["Survived"]
X.head()

,Pclass,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,1,0,7.2500,0,1,0,0,1
1,1,1,0,71.2833,1,0,1,0,0
2,3,0,0,7.9250,1,0,0,0,1
3,1,1,0,53.1000,1,0,0,0,1
4,3,0,0,8.0500,0,1,0,0,1


In [10]:
params_tree = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 1, 3, 5, 10],
    'random_state': [42]
}
grid_tree = GridSearchCV(DecisionTreeClassifier(), params_tree)
grid_tree.fit(X, y)
grid_tree.best_params_, grid_tree.best_score_

({'criterion': 'gini', 'max_depth': 3, 'random_state': 42, 'splitter': 'best'},
 0.8058251208335949)

In [11]:
params_knn = {
    'knn__n_neighbors': [1, 3, 5, 7, 9],
    'knn__weights': ['uniform', 'distance'],
    'knn__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
}
pipe_knn = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier())])
grid_knn = GridSearchCV(pipe_knn, params_knn)
grid_knn.fit(X, y)
grid_knn.best_params_, grid_knn.best_score_

({'knn__algorithm': 'auto', 'knn__n_neighbors': 7, 'knn__weights': 'distance'},
 0.8002761910740066)

In [88]:
# let's submit with tree
tree_best = grid_tree.best_estimator_
test_data_copy = test_data.copy()

X_test = pd.get_dummies(test_data[features])
imputer = SimpleImputer()
imputer.fit(X)
imputed_X_test = pd.DataFrame(imputer.transform(X_test))
imputed_X_test.columns = X_test.columns

predictions = tree_best.predict(imputed_X_test)

In [29]:
# # let's submit with knn
# knn_best = grid_knn.best_estimator_
# test_data_copy = test_data.copy()

# X_test = pd.get_dummies(test_data[features])
# imputer = SimpleImputer()
# imputer.fit(X)
# imputed_X_test = pd.DataFrame(imputer.transform(X_test))
# imputed_X_test.columns = X_test.columns

# predictions = knn_best.predict(imputed_X_test)

In [30]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
